# Find Steady states of rtgM4

In [2]:
using Pkg 
Pkg.activate("SteadyStates")
Pkg.instantiate();

In [10]:
using RetroSignalModel
using Parameters
using FindSteadyStates
using CSV, DataFrames
using ProgressBars
using Random
using SteadyStateDiffEq
using LaTeXStrings
using SteadyStates: mergeCSV, create_null_dataframe, find_undef_name, valid

import RetroSignalModel as rs

## Model

- `rtgM4`

In [26]:
rtgM4 = rs.rtgM4(1)
model = rtgM4.model
param = (;p=rtgM4.p, u=rtgM4.u) 

de = DEsteady(func=model, u0= param.u, p= param.p, method= SSRootfind());

## Find Steady States

Start searching steady states:
1. use `SSRootfind` for both stable and unstable points


### Storage
- `SteadyStates/data/`

In [27]:
savedir =  joinpath("SteadyStates/data/steady_states_")

"SteadyStates/data/steady_states_"

In [3]:
# For Saving to Datafreme. 
if false

    ks = keys(param.u)
    filename = joinpath(save_dir, "steady_states_PLACEHOLDER.csv")

    # number of iteration
    ind = Int(64800)
    iter_number = 10

    for i in ProgressBar(1:iter_number)

        df = create_null_dataframe(ks) # renew df. Clear the memory

        param_ranges = [  
            0.:1.,
            100.:1000.,
            0.:100.,
            0.:1.,
            0.:10.,
            100.:10000.,
            0.:1000.,
            0.:10.,
            0.:100.,
            0.:1000.,
            0.:100.,
            0.:100.,
            0.:10.,
            0.:100.,
            0.:100.,
            0.:100.,
            0.:10.,
        ]

        pr = ParameterRandom(
            param_ranges,
            len = ind)

        sols = solve(de, pr);


        for u in sols.u
            any(isnan, u) ? continue : nothing # skip NaN value
            d = Dict(keys(param.u) .=> u)
            push!(df,d)
        end



        csvname = find_undef_name(filename, "PLACEHOLDER")

        nrow(df) == 0 ? nothing : CSV.write(csvname, df) # skip null dataframe

    end
end 

## Merge files

In [9]:
df = mergeCSV(savedir);

df_ = valid(df; low=0.,high=1e4, s_low=0., s_high=1.0) # valid dataset
sort!(df_)
CSV.write(joinpath("SteadyStates/data", "valid_ss.csv"),df_)

"SteadyStates/data/valid_ss.csv"

In [10]:
df_ = valid(df; low=0.,high=1e4, s_low=0., s_high=1.0) # valid dataset

,Bmh,BmhMks,Mks,Rtg13_a_c,Rtg13_a_n,Rtg13_i_c,Rtg13_i_n,Rtg1_c
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,7308.61,109.906,0.615391,0.0652567,0.173116,7.57005,2.92808,1.52829
2,8647.37,145.583,0.688956,0.759752,2.10235,1470.42,201.717,26.3655
3,2285.03,25.6052,0.458563,0.41964,0.731933,101.543,20.7792,5.46277
4,7048.75,291.425,1.69192,0.527703,1.56541,2982.16,377.687,71.5453
5,7545.26,394.107,2.13749,0.07292,0.220481,3024.23,367.105,515.038
6,5354.27,111.566,0.852701,0.085788,0.228152,21.8763,5.29331,3.51247
7,4584.02,0.567586,0.00506699,5.86037,1.91219,511.751,84.5695,45.9478
8,8938.89,193.247,0.884693,0.088391,0.252937,37.7941,7.4998,5.50624
9,7194.8,227.161,1.29205,0.0798471,0.232118,4.13382,3.19992,0.535395


## Plot Steady States Results in Different Conditions

In [3]:
using Pkg 
Pkg.activate("SteadyStates")
Pkg.instantiate()
using IJulia
using TikzPictures


 Activating environment at `~/project/MitoRetrogradeModel/MitoChannelAnalysis/SteadyStates/Project.toml`


In [5]:
tp = TikzPicture("\\draw (0,0) -- (10,10);\n\\draw (10,0) -- (0,10);\n\\node at (5,5) {tikz \$\\sqrt{\\pi}\$};", options="scale=0.25", preamble="")
save(PDF("test"), tp)

LoadError: SystemError: opening file "./jl_nLWelh/texput.log": No such file or directory

In [21]:

"""
Try conditions of the model with given initial variables and parameters.
"""
function Try_conditions(u,p, VALID_SETTING;
                        BREAK=false,
                        Cond_Random=false)

        @unpack  model, S_SPAN, DEL_CONC, TRANS_THRESHOLD,  SSMETHOD, cond, protein_lookup = VALID_SETTING

        valid = 1

        num_cond = size(cond)[1] # number of conditions

        # Shuffling the conditions
        trial_cond_order = Cond_Random ? shuffle(1:num_cond) : 1:num_cond

        test_log = Dict("nuc"=>[], "cyt"=>[])

        # get initial condition
        de = DEsteady(func=model, u0=u, p=p, method=SSMETHOD)
        #u_ = solve(de) # This step changed the initial value, and reset as the steady ones
        u_ = redist2monomer(u, protein_lookup)

        for trial_n in trial_cond_order # Test conditions

            con = cond[trial_n,:]

            if typeof(con.Trans2Nuc) == Missing 
                push!(test_log["nuc"] , Missing)
                push!(test_log["cyt"], Missing)
                continue  # some conditions are not measured
            end
            

            ud = Get_del_u(u_, con.rtg1, con.rtg2, con.rtg3, con.mks, con.s, protein_lookup; del_conc=DEL_CONC, s_span=S_SPAN)

            sol = solve(de(ud))

            # Check the steady-state is real
            if sol.retcode==:Failure
                valid = 0 #unstable system
                break
            end


            trans2nuc = Get_output(sol, con.gfp, protein_lookup, threshold=TRANS_THRESHOLD) # 1 means nucleus has significantly higer concentration than cytosol

            
  
            push!(test_log["nuc"] , trans2nuc[1])
            push!(test_log["cyt"], trans2nuc[2])
            
 
    end
    
        return test_log
end


function Get_output(sol, gfp, protein_lookup; threshold = TRANS_THRESHOLD)

    if gfp == "rtg1"
        cyt_index = protein_lookup[:Rtg1_c]
        nuc_index = protein_lookup[:Rtg1_n]
    elseif gfp == "rtg3"
        cyt_index = protein_lookup[:Rtg3_c]
        nuc_index = protein_lookup[:Rtg3_n]
    else
        throw(MathodError)
    end

    total_conc_cyt = sum(sol[cyt_index])
    total_conc_nuc = sum(sol[nuc_index])

  
    return (total_conc_nuc, total_conc_cyt)
end

function Get_del_u(u, rtg1, rtg2, rtg3, mks, s, protein_lookup; del_conc=DEL_CONC, s_span=S_SPAN )

    del_info = Dict([:Rtg1, :Rtg2, :Rtg3, :Mks] .=> [rtg1, rtg2, rtg3, mks])
    ud = deepcopy(u)

    for protein in keys(del_info)
        protein_existence = del_info[protein]
        if protein_existence == 0
            ud[protein_lookup[protein]] .= del_conc
        end
    end

    ud[protein_lookup[:s]] = s_span[s+1] #in boolean, s=0 means 1 in span_tuple

    return ud
end


function redist2monomer(u, protein_lookup)
    u_dist = zeros(length(u))
    
    
    # Rtg2 
    sum_rtg2 = sum(u[protein_lookup[:Rtg2]])
    u_dist[2] = sum_rtg2
    
     # Rtg3
    sum_rtg3 = sum(u[protein_lookup[:Rtg3]])
    u_dist[11] = sum_rtg3
    
     # Rtg1
    sum_rtg1 = sum(u[protein_lookup[:Rtg1]])
    u_dist[14] = sum_rtg1
    
    # Mks
    sum_mks = sum(u[protein_lookup[:Mks]])
    u_dist[4] = sum_mks
    
    # Bmh 
    sum_bmh = sum(u[protein_lookup[:Bmh]])
    u_dist[6] = sum_bmh
    
    return u_dist
    
end

param = RetroSignalModel.rtgM4.param();
model = RetroSignalModel.rtgM4.model;
u = param.u; 
p = param.p;

In [26]:
u_sep = redist2monomer(u,RetroSignalModel.get_protein_lookup(model))
for (j,i) in zip(u_sep, model.states)
    println(i,"  ",j)
end

s(t)  0.0
Rtg2_ina_c(t)  651.6543766184084
Rtg2_act_c(t)  0.0
Mks(t)  267.49672022463324
Rtg2Mks_c(t)  0.0
Bmh(t)  10000.000000000005
BmhMks(t)  0.0
Rtg13_a_c(t)  0.0
Rtg13_i_c(t)  0.0
Rtg3_i_c(t)  0.0
Rtg3_a_c(t)  409.5036119882923
Rtg3_a_n(t)  0.0
Rtg3_i_n(t)  0.0
Rtg1_c(t)  145.92035635683752
Rtg1_n(t)  0.0
Rtg13_a_n(t)  0.0
Rtg13_i_n(t)  0.0


In [27]:
transRes = Try_conditions(u_sep,p, valid_setting(model))

Dict{String,Array{Any,1}} with 2 entries:
  "cyt" => Any[Missing, 43.3458, 43.3458, 43.3451, 193.737, 193.737, 193.737, 6…
  "nuc" => Any[Missing, 366.158, 366.158, 366.159, 215.766, 215.766, 215.766, 3…

In [28]:
df = DataFrame(transRes)

,cyt,nuc
,Any,Any
1,Missing,Missing
2,43.3458,366.158
3,43.3458,366.158
4,43.3451,366.159
5,193.737,215.766
6,193.737,215.766
7,193.737,215.766
8,63.2278,346.276
9,Missing,Missing


In [29]:
Rtg2 = sum(u[RetroSignalModel.get_protein_lookup(model)[:Rtg2]])

651.6543766184084

In [30]:
RetroSignalModel.RTG_Response_Boolean

,rtg1,rtg2,rtg3,s,mks,gfp,Trans2Nuc
,Int64,Int64,Int64,Int64,Int64,String,Int64?
1,0,0,1,0,1,rtg3,missing
2,0,0,1,1,1,rtg3,1
3,0,1,1,0,1,rtg3,1
4,0,1,1,1,1,rtg3,1
5,1,0,1,0,1,rtg3,0
6,1,0,1,1,1,rtg3,0
7,1,1,1,0,1,rtg3,0
8,1,1,1,1,1,rtg3,1
9,1,0,0,0,1,rtg1,missing


In [37]:
TexPic = L"""
\documentclass[border={5pt 0pt 20pt 5pt}, preview]{standalone}

\usepackage{pgfplots}
\pgfplotsset{compat=1.15}
\usepackage{xcolor}


\usepackage{tikz}
\usepackage[utf8]{inputenc}
\usepackage{xfp}
\usepackage{booktabs}
\usetikzlibrary{calc}
\usepackage{graphicx}

\newcommand{\Del}[1]{\textit{$\Delta$\MakeLowercase{#1}}}
\newcommand\ratio[2]{%
    \pgfmathparse{ #1/(#1 + #2)}\pgfmathresult
}


\newcommand\colorBar{
    \begin{tikzpicture}[baseline={(0,0)}, inner sep=0,outer sep=0]

       
        \pgfdeclarehorizontalshading{someShading}{4cm}{
        color(0cm)=(green!0!gray);
        color(4cm)=(green!100!gray)
        }
        
        \shade [shading=someShading, yshift=-1cm]  (0,0) rectangle ++(1,0.3);
        \node [] at (0,-1.3) {0};
        \node [] at (1,-1.3) {1};
        
        \end{tikzpicture}
}


\newcommand\particle[2]{%
  \begin{tikzpicture}[x=2cm, y=2cm]
  

  % Calculate the ratio of nucleus to cytosol concentration
  \def\nratio{\fpeval{ round(#1/(#1 + #2),2)}  };
  \def\cratio{\fpeval{ round(#2/(#1 + #2),2)}  };

  \def\nconc{\fpeval{ round(#1,1)}  };
  \def\cconc{\fpeval{ round(#2,1)}  };
  
  %Node for Circles
  \node at (0,0) [circle,draw, scale=1 ,fill=green!\fpeval{round(100*(\cratio))}!gray] (c2) {};
  
  \node at (0,0) [circle,draw, scale=0.5 ,fill=green!\fpeval{round(100*(\nratio))}!gray] (c1) {};
  
  
  %Node for line ends
  \node [inner sep=0,outer sep=0, xshift=0.2cm] at (c1.east) (n_conc) {};
  \node [inner sep=0,outer sep=0,yshift = -0.09cm] at (n_conc.south) (c_conc) {};
  
  % Node for data
  \node [inner sep=0,outer sep=0, xshift=0.cm, text width = 0cm, align=left] at (n_conc.east) {\scalebox{.2}{\nconc} };
  \node [inner sep=0,outer sep=0, xshift=0.cm, text width = 0cm, align=left] at (c_conc.east) {\scalebox{.2}{\cconc}};
  
  % Draw Lines
  \draw[-, very thin] (n_conc) -- (c1);
  \draw[-, very thin] (c_conc) -- (0.16cm,-0.1065cm);
  
  \end{tikzpicture}
}

\newcommand\cell[2]{
    \begin{tikzpicture}[baseline={(c_conc.base)}, x=2cm, y=2cm, , outer sep=0]
  
        %Node for Circles
        \node at (0,0) [circle,draw, scale=1] (c2) {};
        
        \node at (0,0) [circle,draw, scale=0.5] (c1) {};
        
        
        %Node for line ends
        \node [inner sep=0,outer sep=0, xshift=0.2cm] at (c1.east) (n_conc) {};
        \node [inner sep=0,outer sep=0,yshift = -0.09cm] at (n_conc.south) (c_conc) {};
        
        % Node for data
        \node [inner sep=0,outer sep=0, xshift=0.cm, text width = 0cm, align=left] at (n_conc.east) {\scalebox{.2}{#1} };
        \node [inner sep=0,outer sep=0, xshift=0.cm, text width = 0cm, align=left] at (c_conc.east) {\scalebox{.2}{#2}};
        
        % Draw Lines
        \draw[-, very thin] (n_conc) -- (c1);
        \draw[-, very thin] (c_conc) -- (0.16cm,-0.1065cm);
        \end{tikzpicture}
    
}

\newcommand\drawcell[2]{%
    \begin{tikzpicture}[baseline={(c_conc.base)}, outer sep=0]
        \def\sc{3}
        \node at (0,0) [scale=\sc] {\cell{#1}{#2}};
    \end{tikzpicture}
}

\newcommand\drawRatio[2]{%
    \begin{tikzpicture}[baseline={(c_conc.base)}, outer sep=0]
        \def\sc{3}
        \node at (0,0) [scale=\sc] {\particle{#1}{#2}};
    \end{tikzpicture}
}



\begin{document}

\begin{tabular}[c]{p{1.4cm}p{1.8cm}p{2.8cm}|p{1.8cm}p{1.6cm}}
    \toprule
    \multicolumn{1}{c}{} & \multicolumn{2}{c}{\textbf{Healthy Mitochondria}} & \multicolumn{2}{c}{\textbf{Damaged Mitochondria}} \\ \midrule
     &\textbf{Simulation} & \multicolumn{1}{c}{\textbf{Data}}\hspace{0.52cm}  & \textbf{Simulation} & \hspace{0.5cm}\textbf{Data}  \\ 
     &   \multicolumn{4}{c}{\textbf{Rtg3-GFP}}  \\
    WT & \drawRatio{%$(transRes["nuc"][7])}{%$(transRes["cyt"][7])} & \drawRatio{0}{1} & \drawRatio{%$(transRes["nuc"][8])}{%$(transRes["cyt"][8])} & \drawRatio{1}{0} \\
    \Del{Rtg1} & \drawRatio{%$(transRes["nuc"][3])}{%$(transRes["cyt"][3])} & \drawRatio{1}{0}  & \drawRatio{%$(transRes["nuc"][4])}{%$(transRes["cyt"][4])} & \drawRatio{1}{0} \\
    \Del{Rtg2} & \drawRatio{%$(transRes["nuc"][5])}{%$(transRes["cyt"][5])} & \drawRatio{0}{1}& \drawRatio{%$(transRes["nuc"][6])}{%$(transRes["cyt"][6])} & \drawRatio{0}{1} \\
    \Del{Mks} & \drawRatio{%$(transRes["nuc"][18])}{%$(transRes["cyt"][18])} & \drawRatio{1}{0}& \multicolumn{2}{r}{}  \\
    \Del{Rtg2}\Del{Mks} & \drawRatio{%$(transRes["nuc"][17])}{%$(transRes["cyt"][3])} & \drawRatio{1}{0} & \multicolumn{2}{r}{} \\ 
     &   \multicolumn{4}{c}{\textbf{Rtg1-GFP}}   \\
    WT & \drawRatio{%$(transRes["nuc"][15])}{%$(transRes["cyt"][15])} & \drawRatio{0}{1} & \drawRatio{%$(transRes["nuc"][16])}{%$(transRes["cyt"][16])} & \drawRatio{1}{0} \\
    \Del{Rtg3} & \drawRatio{%$(transRes["nuc"][11])}{%$(transRes["cyt"][11])} & \drawRatio{0}{1}& \drawRatio{%$(transRes["nuc"][12])}{%$(transRes["cyt"][12])} & \drawRatio{0}{1} \\
    \Del{Rtg2} & \drawRatio{%$(transRes["nuc"][13])}{%$(transRes["cyt"][13])} & \drawRatio{0}{1} & \drawRatio{%$(transRes["nuc"][14])}{%$(transRes["cyt"][14])} & \drawRatio{0}{1} \\ 
    \Del{Mks} & \drawRatio{%$(transRes["nuc"][20])}{%$(transRes["cyt"][20])} & \drawRatio{1}{0}& \multicolumn{2}{r}{}  \\
    \Del{Rtg2}\Del{Mks} & \drawRatio{%$(transRes["nuc"][19])}{%$(transRes["cyt"][19])} & \drawRatio{1}{0} & \multicolumn{2}{r}{} \\ 
      &   \multicolumn{4}{c}{\textbf{Rtg2-GFP}}   \\
    WT & \drawRatio{0}{%$Rtg2} & \drawRatio{0}{1} & \drawRatio{0}{%$Rtg2} & \drawRatio{0}{1} \\
    &  \multicolumn{4}{c}{
        
    \begin{tikzpicture}
        \node [] (cell) {\drawcell{Nucleus}{Cytosol}};
        \node [ right of=cell, xshift=1.5cm] (col) {\colorBar};
    \end{tikzpicture}
    } 
    
    \\
    \bottomrule
\end{tabular}

\end{document}
"""



L"\documentclass[border={5pt 0pt 20pt 5pt}, preview]{standalone}

\usepackage{pgfplots}
\pgfplotsset{compat=1.15}
\usepackage{xcolor}


\usepackage{tikz}
\usepackage[utf8]{inputenc}
\usepackage{xfp}
\usepackage{booktabs}
\usetikzlibrary{calc}
\usepackage{graphicx}

\newcommand{\Del}[1]{\textit{$\Delta$\MakeLowercase{#1}}}
\newcommand\ratio[2]{%
    \pgfmathparse{ #1/(#1 + #2)}\pgfmathresult
}


\newcommand\colorBar{
    \begin{tikzpicture}[baseline={(0,0)}, inner sep=0,outer sep=0]

       
        \pgfdeclarehorizontalshading{someShading}{4cm}{
        color(0cm)=(green!0!gray);
        color(4cm)=(green!100!gray)
        }
        
        \shade [shading=someShading, yshift=-1cm]  (0,0) rectangle ++(1,0.3);
        \node [] at (0,-1.3) {0};
        \node [] at (1,-1.3) {1};
        
        \end{tikzpicture}
}


\newcommand\particle[2]{%
  \begin{tikzpicture}[x=2cm, y=2cm]
  

  % Calculate the ratio of nucleus to cytosol concentration
  \def\nratio{\fpeval{ round(#1/(#1 + #2),2)}  

In [45]:
open("SteadyStates/result/verification/PLOT_SteadyStateVerification.tex", "w") do io
           write(io, TexPic)
       end

5536

# ;cd SteadyStates/result/verification/ 

In [34]:
;xelatex PLOT_SteadyStateVerification  -synctex=1

This is XeTeX, Version 3.14159265-2.6-0.99998 (TeX Live 2017/Debian) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
! I can't find file `PLOT_SteadyStateVerification'.
<*> PLOT_SteadyStateVerification 
                                 -synctex=1
(Press Enter to retry, or Control-D to exit)
Please type another input file name: 
! Emergency stop.
<*> PLOT_SteadyStateVerification 
                                 -synctex=1
No pages of output.
Transcript written on texput.log.


In [35]:
;cd ../../../

/home/ubuntu


In [36]:
;pwd

/home/ubuntu
